In [1]:
using CSV, Tables, LinearAlgebra, Random, Gurobi, JuMP, DataFrames

In [2]:
df = DataFrame(CSV.File("final_data.csv"));

In [3]:
data = CSV.File("final_data.csv", header=true) |> Tables.matrix;

In [4]:
#Part d
function stable_regression(X, y, k)
    m = Model(Gurobi.Optimizer)
    set_optimizer_attribute(m, "OutputFlag", 0)
    
    n, p = size(X)
    
    @variable(m, B[1:p])
    @variable(m, u[1:n])
    @variable(m, θ)
    #println(typeof(B))
    #println(typeof(u))
    #println(typeof(y))
    #println(typeof(X))
    @constraint(m, u .+ θ .>= (y - X*B).^2)
    @constraint(m, u .>= 0)
    @objective(m, Min, k*θ + sum(u))

    optimize!(m)
    return value.(B)
end;

In [5]:
y_solar = data[:,11];
X = data[:,13:38];
y_wind = data[:,12];


In [6]:
function MSE(y, pred) 
    return sum((y-pred).^2)/length(y)
end;

### SOLAR: BEST SPLIT
60% TRAIN 40% VALIDATION

**We pick the split with the worst MSE in stable regression**

In [7]:
n,p = size(X)
MSE_best_solar, k_best_solar = 0, 0
best_beta_solar = zeros()
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_solar[1:k]
    X_valid = Matrix(X[k+1:n,:])
    y_valid = y_solar[k+1:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp >= MSE_best_solar
        MSE_best_solar = MSE_temp
        k_best_solar = k 
        best_beta_solar = β_temp
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


In [8]:
k_best_solar #best split is 60% training and 40% validation

5255

In [9]:
MSE_best_solar

93810.12380398327

##### Pulling out the indices with the worst residuals to determine the training data

In [10]:
solar_residuals = (y_solar.-X*best_beta_solar).^2;
solar_sorted_resid = sort(solar_residuals, rev = true);

In [11]:
train_indices = []
test_indices = []
for ind = 1:8759
    if solar_residuals[ind] in solar_sorted_resid[1:5255]
        push!(train_indices,ind)
    else
        push!(test_indices,ind)
    end 
end



In [12]:
X_best_solar = zeros(5255,26)
y_best_solar = zeros(5255)
for numb =1:5255
    X_best_solar[numb,:] = X[train_indices[numb],:]
    y_best_solar[numb] = y_solar[train_indices[numb]]
end

In [13]:
X_valid_solar = zeros(3504,26)
y_valid_solar = zeros(3504)
for numb =1:3504
    X_valid_solar[numb,:] = X[test_indices[numb],:]
    y_valid_solar[numb] = y_solar[test_indices[numb]]
end

In [14]:
X_train_solar = X_best_solar
X_valid_solar = X_valid_solar
y_train_solar = y_best_solar
y_valid_solar = y_valid_solar;

In [15]:
#transform back to dataframe to understand the coefficients
X_train_solar_df=DataFrame(X_train_solar, :auto)
X_valid_solar_df=DataFrame(X_valid_solar, :auto)
#rename columns with list of values
rename!(X_train_solar_df, names(df[:, 13:38]))
rename!(X_valid_solar_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_solar=DataFrame(Solar_Gen_MW = y_train_solar)
y_valid_solar=DataFrame(Solar_Gen_MW = y_valid_solar)
#save as CSV
CSV.write("X_train_solar.csv", X_train_solar_df)
CSV.write("X_valid_solar.csv", X_valid_solar_df)
CSV.write("y_train_solar.csv", y_train_solar)
CSV.write("y_valid_solar.csv", y_valid_solar);

In [17]:
vcat(X_train_solar_df, X_valid_solar_df)[8700:8759,:]

,temp,feelslike,dew,humidity,precip,precipprob,snow,snowdepth
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,52.1,52.1,48.8,88.62,0.0,0.0,0.0,0.0
2,53.1,53.1,50.0,89.08,0.0,0.0,0.0,0.0
3,57.0,57.0,51.5,81.89,0.0,0.0,0.0,0.0
4,62.1,62.1,52.3,70.28,0.0,0.0,0.0,0.0
5,64.9,64.9,51.9,62.84,0.0,0.0,0.0,0.0
6,65.9,65.9,50.1,56.68,0.0,0.0,0.0,0.0
7,71.8,71.8,47.5,42.07,0.0,0.0,0.0,0.0
8,71.2,71.2,52.8,52.24,0.0,0.0,0.0,0.0
9,69.9,69.9,50.0,49.22,0.0,0.0,0.0,0.0


In [18]:
X_total_solar = vcat(X_train_solar_df, X_valid_solar_df)
y_total_solar = vcat(y_train_solar,y_valid_solar);

CSV.write("X_total_solar.csv", X_total_solar)
CSV.write("y_total_solar.csv", y_total_solar);

### WIND: BEST SPLIT 
70% TRAIN 30% VALIDATION

In [19]:
n,p = size(X)
MSE_best_wind, k_best_wind = 0, 0
B_best_wind = zeros(26)
X_train_best, X_valid_best, y_train_best, y_valid_best = nothing, nothing, nothing, nothing
for k in [4380,4818,5255,5693,6131,6569,7007,7445,7883] #we look for the best split of the data
    X_train = Matrix(X[1:k,:])
    y_train = y_wind[1:k]
    X_valid = Matrix(X[k:n,:])
    y_valid = y_wind[k:n]

    β_temp = stable_regression(X_train, y_train, k)
    MSE_temp = MSE(y_valid, X_valid*β_temp)
    if MSE_temp >= MSE_best_wind
        MSE_best_wind = MSE_temp
        k_best_wind = k 
        B_best_wind = β_temp
        X_train_best = X_train
        X_valid_best = X_valid
        y_train_best = y_train
        y_valid_best = y_valid
    end
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18
Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-18


In [20]:
println(k_best_wind) #best split is 70% training and 30% validation
println(MSE_best_wind)

6131
1.4664152769031666e7


In [21]:
wind_residuals = (y_wind.-X*B_best_wind).^2;
wind_sorted_resid = sort(wind_residuals, rev = true);

In [22]:
wind_train_indices = []
wind_test_indices = []
for ind = 1:8759
    if wind_residuals[ind] in wind_sorted_resid[1:6131]
        push!(wind_train_indices,ind)
    else
        push!(wind_test_indices,ind)
    end 
end


In [23]:
X_best_wind= zeros(6131,26)
y_best_wind= zeros(6131)
for numb =1:6131
    X_best_wind[numb,:] = X[wind_train_indices[numb],:]
    y_best_wind[numb] = y_wind[wind_train_indices[numb]]
end

In [24]:
X_valid_wind = zeros(2628,26)
y_valid_wind = zeros(2628)
for numb =1:2628
    X_valid_wind[numb,:] = X[wind_test_indices[numb],:]
    y_valid_wind[numb] = y_solar[wind_test_indices[numb]]
end

In [25]:
X_train_wind = X_best_wind
y_train_wind = y_best_wind
X_valid_wind = X_valid_wind
y_valid_wind = y_valid_wind;

In [26]:
#transform back to dataframe to understand the coefficients
X_train_wind_df=DataFrame(X_train_wind, :auto)
X_valid_wind_df=DataFrame(X_valid_wind, :auto)
#rename columns with list of values
rename!(X_train_wind_df, names(df[:, 13:38]))
rename!(X_valid_wind_df, names(df[:, 13:38]))

#transform vector into dataframe 
y_train_wind=DataFrame(Wind_Gen_MW = y_train_wind)
y_valid_wind=DataFrame(Wind_Gen_MW = y_valid_wind)
#save as CSV
CSV.write("X_train_wind.csv", X_train_wind_df)
CSV.write("y_train_wind.csv", y_train_wind)
CSV.write("X_valid_wind.csv", X_valid_wind_df)
CSV.write("y_valid_wind.csv", y_valid_wind);

In [27]:
X_total_wind = vcat(X_train_wind_df, X_valid_wind_df)
y_total_wind = vcat(y_train_wind,y_valid_wind);

In [28]:
CSV.write("X_total_wind.csv", X_total_wind)
CSV.write("y_total_wind.csv", y_total_wind);

In [26]:
using Statistics
SS_t = sum((y_solar .- mean(y_solar)).^2)

# Regression sum of squares
SS_r = sum(((X*best_beta_solar) .- mean(y_solar)).^2)

R2_wind = SS_r/ SS_t

0.15677170606012225

In [27]:
SS_t_wind = sum((y_wind .- mean(y_wind)).^2)

# Regression sum of squares
SS_r_wind = sum(((X*B_best_wind) .- mean(y_wind)).^2)

R2_wind = SS_r_wind/ SS_t_wind

0.47151454919040925